# MathematicalProgram 디버깅 팁(debugging tips)
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.

## 중요 공지(Important Note)
Drake에서 일반적인 최적화 프로그램을 구성하고 해결하는 방법은 수학적 프로그램 튜토리얼[mathematical program tutorial](./mathematical_program.ipynb)을 참조하자.

Drake의 `MathematicalProgram` 인터페이스를 통해 최적화 문제를 구성하고 해결한 후에도 원하는 결과를 얻지 못할 수 있다. 예를 들어, 문제에 해가 있다고 예상했는데 MathematicalProgram이 문제가 성공적으로 해결되지 않았다고 보고하는 경우가 있다. 이 튜토리얼에서는 `MathematicalProgram`이 원하는 대로 작동하지 않을 때 디버깅하는 몇 가지 팁에 대해서 알아보자.

먼저 최적화 문제가 볼록(convex) 인지 여부를 이해해야 한다. 볼록 문제(convex problem : LP, QP, SDP 등)의 경우 문제가 가능할 때 이론적으로 솔루션을 항상 찾을 수 있어야 한다. 반면에 문제가 비볼록(non-convex)하고 경사 기반 솔버(SNOPT/IPOPT 등)를 통해 해결되는 경우 해가 존재하더라도 솔버가 가능한 해에서 종료되지 않을 수 있다. 경사 기반 솔버(gradient-based solvers : SNOPT/IPOPT)가 문제가 불가능하다고 보고하는 경우는 솔버가 불가능한 값에 갇혀 있다는 것만 의미하며 로컬 영역에서 불가능성을 줄이는 방법을 모르는 경우이다. 솔버가 막힌 곳에서 멀리 떨어진 곳에 해가 존재할 수 있지만 솔버는 갇혀 있기 때문에 멀리 있는 해로 점프할 수 없다. 한 가지 해결책은 최적화 프로그램의 다른 초기 추측값을 선택하는 것이다.

다음은 비선형 최적화(nonlinear optimization)에서 초기 추측값의 중요성을 보여주는 예제이다.

In [ ]:
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from pydrake.solvers import MathematicalProgram
from pydrake.solvers import IpoptSolver
import numpy as np


def constraint(x):
    return [np.cos(x[0]) + 2 * np.cos(x[0] - x[1])]

# Find a solution satisfying
# 1 <= cos(x[0]) + 2 * cos(x[0] - x[1]) <= 2
# This problem has infinitely many solutions, for example, x = [0, pi/2]

# To visualize the constraint, I draw the landscape of cos(x[0]) + 2 * cos(x[0] - x[1])), and
# also highlight the point x = [0, 0]. You could see that x = [0, 0] is at a
# peak of the landscape.
def draw_constraint_landscape():
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x_mesh, y_mesh = np.meshgrid(np.linspace(-np.pi, np.pi, 31), np.linspace(-np.pi, np.pi, 31))
    constraint_val = np.cos(x_mesh) + 2 * np.cos(x_mesh - y_mesh)
    surf = ax.plot_surface(x_mesh, y_mesh, constraint_val, cmap=cm.coolwarm, alpha=0.8)
    ax.plot([0], [0], [3], marker='.', color='g', markersize=20)
    ax.set_xlabel("x[0]")
    ax.set_ylabel("x[1]")
    ax.set_zlabel("cos(x[0]) + 2 * cos(x[0]-x[1])")
    fig.show()
    

draw_constraint_landscape()

prog = MathematicalProgram()
x = prog.NewContinuousVariables(2, "x")

prog.AddConstraint(constraint, [1], [2], x)

solver = IpoptSolver()
# With the initial guess being (0, 0), the solver cannot find a solution.
prog.SetInitialGuess(x, [0, 0])
result = solver.Solve(prog)
print(f"Starting from x=[0, 0], the solver result is {result.get_solution_result()}")
print(f"The solver gets stuck at x={result.GetSolution(x)}")

# With a different initial guess, the solver can find the solution
prog.SetInitialGuess(x, [0.1, 0.5])
result = solver.Solve(prog)
print(f"Starting from x=[0.1, 0.5], the solver result is {result.get_solution_result()}")
print(f"The found solution is x={result.GetSolution(x)}")

위의 예에서, 나쁜 초기 추측 $x = [0, 0]$으로 인해 솔버가 멈추고 문제가 실현 불가능하다고 보고하는 것을 볼 수 있습니다(멈추는 이유는 제약 조건 함수 $cos(x[0]) + 2cos(x[0]-x[1])$의 기울기가 초기 추측 $x=[0, 0]$에서 0이기 때문에 경사 기반 솔버는 결정 변수(decision variables)를 어떻게 이동해야 하는지 알 수 없다. 이 현상은 특이점에서 초기 자세로 역기구학 문제를 풀거나 초기 추측 $x=0$으로 단위 길이 제약 $x^Tx=1$을 풀 때도 나타납니다. 하지만 초기 추측을 바꾸면 솔버가 해를 찾을 수 있다.

솔버(SNOPT와 같은)가 최적화 프로세스 중에 실현 가능한 솔루션을 찾았더라도 다음 반복에서 실현 불가능한 값으로 점프할 수 있다. SNOPT는 최적화 프로세스 중에 있는 경우 실현 가능한 영역 내에 머무른 다는 것을 보장하지 않는다.

가끔 제약 조건이 잘못 부과되어 문제가 실현 불가능할 수 있다. 최적화가 실패하는 이유를 이해하기 위해 몇 가지 디버깅 팁을 제공한다. 이러한 팁을 사용하여 문제가 있는 제약 조건/초기 추측을 진단할 수 있다.

## MathematicalProgram의 요약을 출력하기(Print a summary of the MathematicalProgram)
특히 작은 문제의 경우, MathematicalProgram을 문자열로 표시하면 매우 유용할 수 있다. 이를 통해 프로그램에 추가된 결정 변수, 비용, 제약 조건 목록을 확인할 수 있다.

In [ ]:
# A sample (quadratic) program
prog = MathematicalProgram()
x = prog.NewContinuousVariables(3, "x")
prog.AddQuadraticCost(x[0] * x[0] + 2 * x[0] + 3)
prog.Add2NormSquaredCost(A = [[1, 3], [2, 4]], b=[1, 4], vars=[x[1], x[2]])
prog.AddLinearEqualityConstraint(x[0] + 2*x[1] == 5)
prog.AddLinearConstraint(x[0] + 4 *x[1] <= 10)
prog.AddBoundingBoxConstraint(-1, 10, x)

# Now print a summary:
print(prog)

LaTeX로 program을 표시할 수 있다.

In [ ]:
display(Markdown(prog.ToLatex()))

## costs/constraints/variables에 이름 붙이기(Name your costs/constraints/variables)
진단 목적으로 몇 가지 제약/비용을 출력하면 종종 도움이 된다. 의미 있는 출력 메시지를 얻으려면 비용/제약/변수에 이름을 지정할 수 있다.
### 1. 변수 이름 붙이기(Name the variables)
`NewContinuousVariables`(또는 `NewBinaryVariables`)를 통해 변수를 생성할 때 변수 이름으로 문자열을 전달할 수 있다. 다음은 예시이다.

In [ ]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2, "point")
print(x)

### 2. 제약에 이름 붙이기(Name the constraint)
`set_description()` 함수를 사용하여 제약 조건에 이름을 지정할 수 있다. 다음은 예시이다.

In [ ]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2, "point")
constraint = prog.AddConstraint(lambda z: [np.sum(z**2)], [1.], [1.], x)
constraint.evaluator().set_description("unit-length constraint")
print(constraint)

### 3. cost에 이름 붙이기(Name the cost)
유사하게 `set_description()` 함수를 사용하여 cost에 이름을 지정할 수 있다. 다음은 예시이다.

In [ ]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2, "point")
# Add the cost on the distance to (1, 2)
cost1 = prog.AddCost(lambda z: np.sqrt((z[0]-1)**2 + (z[1]-2)**2), x)
cost1.evaluator().set_description("distance to (1, 2)")
# Add the cost on the distance to (3, -1)
cost2 = prog.AddCost(lambda z: np.sqrt((z[0]-3)**2 + (z[1] + 1)**2), x)
cost2.evaluator().set_description("distance to (3, -1)")
print(f"cost1: {cost1}")
print(f"cost2: {cost2}")

다음 섹션에서 살펴보겠지만, 실현 불가능한 제약 조건을 출력할 수 있다. 변수/제약/비용에 이름을 지정하면 출력 메시지가 더 의미 있는 메시지가 된다.

## Call GetInfeasibleConstraints()
`MathematicalProgram`이 SNOPT/IPOPT와 같은 경사 기반 솔버를 통해 해결되고 문제가 실현 불가능하다고 보고하는 경우, 솔버는 중단된 지점의 결정 변수 값을 반환합니다. [MathematicalProgramResult::GetInfeasibleConstraints()](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.MathematicalProgramResult.GetInfeasibleConstraints) 또는 [MathematicalProgramResult::GetInfeasibleConstraintNames()](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.MathematicalProgramResult.GetInfeasibleConstraintNames)를 호출하여 해당 변수 값에서 큰 위반이 있는 제약 조건을 검색할 수 있다. 그런 다음 검색된 실현 불가능한 제약 조건을 진단하고 그에 따라 제약 조건/초기 추측을 개선할 수 있다.

다음은 예시이다.

In [ ]:
import numpy as np
from pydrake.autodiffutils import (
    InitializeAutoDiff,
    ExtractGradient,
)
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2, "x")

# Add the constraint dist(x, 0) >= 1
constraint1 = prog.AddConstraint(lambda z: [z.dot(z)], [1], [np.inf], x)
constraint1.evaluator().set_description("outside unit circle")

# Add the constraint x[0]**2 + 4 * x[1]**2 <= 4
constraint2 = prog.AddConstraint(lambda z: [z[0]**2 + 4 * z[1]**2], [0], [4], x)
constraint2.evaluator().set_description("inside ellipsoid 1")

solver = IpoptSolver()
prog.SetInitialGuess(x, [0, 0])
result = solver.Solve(prog)
print("Start from initial guess x = [0, 0]")
print(f"optimization status: {result.get_solution_result()}")
infeasible_constraints = result.GetInfeasibleConstraints(prog)
for c in infeasible_constraints:
    print(f"infeasible constraint: {c}")
x_stuck = result.GetSolution(x)
print(f"x_stuck={x_stuck.T}")
# Now evaluate the gradient of the constraint at x_stuck (where the solver gets stuck)
print(f"Gradient of the infeasible constraint at x_stuck: {ExtractGradient(infeasible_constraints[0].evaluator().Eval(InitializeAutoDiff(x_stuck)))}")

# For a different initial state, the constraint that was infeasible now has non-zero gradient
x_new = np.array([0.1, 0.2])
print(f"\nStart from initial guess x_new = {x_new}")
print(f"Gradient of the infeasible constraint at x_new: {ExtractGradient(infeasible_constraints[0].evaluator().Eval(InitializeAutoDiff(x_new)))}")
prog.SetInitialGuess(x, x_new)
# With this new initial guess, the solver will be able to find the solution
result = solver.Solve(prog)
print(f"optimization status: {result.get_solution_result()}")

## solver verbosity 활성화(Enable solver verbosity)
많은 솔버는 각 반복에서 진행 상황을 출력할 수 있다. 출력을 활성화하려면 [SolverOptions](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.SolverOptions)에서 [kPrintToConsole](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.CommonSolverOption) 또는 [kPrintFileName](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.CommonSolverOption)을 활성화한다.

Jupyter nootebook에서 코드를 실행할 때 솔버는 notebook 창이 아닌 notebook이 시작된 콘솔에 출력된다는 점에 유의하자. online notebook(예: Deepnote)의 경우 콘솔 출력이 표시되지 않습니다. 이 경우 파일로 출력하는 것이 좋습니다.

In [ ]:
from pydrake.solvers import CommonSolverOption, SolverOptions

# Create a simple program consisting of
#  constraint: 1 <= squared_norm(x) <= 2.
#  constraint: 2 <= x[0]**2 + 4 * x[0]*x[1] + 4*x[1]**2 + 2 * x[0] <= 5.
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddConstraint(lambda z: [z.dot(z)], [1], [2], x)
prog.AddConstraint(lambda z: [z[0] ** 2 + 4 * z[0] * z[1] + 4 * z[1]**2 + 2 * z[0]], [2], [5], x)
prog.SetInitialGuess(x, [0, 0])

# Solve with printing.
ipopt_solver = IpoptSolver()
filename = "/tmp/debug.txt"
solver_options = SolverOptions()
solver_options.SetOption(CommonSolverOption.kPrintFileName, filename)
result = ipopt_solver.Solve(prog, solver_options=solver_options)
with open(filename) as f:
    print(f.read())


일부 솔버는 진행 상황 출력에 대한 더 세밀한 미세 설정 기능을 제공한다. 예를 들어, [IPOPT options](https://coin-or.github.io/Ipopt/OPTIONS.html)은 콘솔 출력을 위한 `print_level` 또는 파일 출력을 위한 `file_print_level`이라는 IPOPT 고유의 옵션을 제공한다.

In [ ]:
# Enable a slightly more verbose level of printing this time.
# The options can be attached to the program, instead of passed to Solve().
solver_options.SetOption(IpoptSolver.id(), "print_level", 5)
prog.SetSolverOptions(solver_options)
result = ipopt_solver.Solve(prog)

## Callback 추가하기(Add Callback)
일부 솔버는 callback 함수를 추가하는 기능을 지원하며, 이 함수는 최적화 프로세스의 각 반복마다 실행된다. 이 callback을 사용하여 진행 상황을 시각화할 수 있다.

In [ ]:
# Visualize the solver progress in each iteration through a callback
# Find the closest point on a curve to a desired point.
from pydrake.solvers import Solve

fig = plt.figure()
curve_x = np.linspace(1, 10, 100)
ax = plt.gca()
ax.plot(curve_x, 9./curve_x)
ax.plot(-curve_x, -9./curve_x)
ax.plot(0, 0, 'o')
x_init = [4., 5.]
ax.plot(x_init[0], x_init[1], 'x')
ax.axis('equal')

def visualization_callback(x):
    ax.plot(x[0], x[1], 'x', color='gray')
    
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddConstraint(x[0] * x[1] == 9)
prog.AddCost(x[0]**2 + x[1]**2)
prog.AddVisualizationCallback(visualization_callback, x)
result = Solve(prog, x_init)

## EvalBinding 사용하기(Use EvalBinding)
개별 제약 조건/비용을 각각 평가하기 위해 프로그램 결정 변수를 `x`로 설정한 상태에서 제약 조건/비용 `binding`을 평가하려면 [EvalBinding(binding, x)](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.MathematicalProgram.EvalBinding)를 호출할 수 있다.

In [ ]:
## Demonstrate EvalBinding function
prog = MathematicalProgram()
p1 = prog.NewContinuousVariables(2, "p1")
p2 = prog.NewContinuousVariables(2, "p2")

# Add the constraint that p1 is in an ellipsoid (p1(0)-1)**2 + 4*(p1(1)-2)**2 <= 1
constraint1 = prog.AddConstraint(lambda z: [(z[0]-1)**2 + 4 * (z[1]-2)**2], [0], [1], p1)
# Add the constraint that p2 is in an ellipsoid (p2(0) + 2)**2 + 0.25*(p2(1)+ 1)**2) <= 1
constraint2 = prog.AddConstraint(lambda z: [(z[0]+2)**2 + 0.25*(z[1]+1)**2], [0], [1], p2)
# Add a cost to minimize the distance between p1 and p2
cost = prog.AddCost((p1-p2).dot(p1-p2))

# Evaluate the constraint and cost at a guess p1=[0, 1], p2 = [-1, -4]
p1_val = [0, 1]
p2_val = [-1, -4]
prog.SetInitialGuess(p1, p1_val)
prog.SetInitialGuess(p2, p2_val)
print(f"constraint 1 evaluated at p1={p1_val}, p2={p2_val} is {prog.EvalBinding(constraint1, prog.initial_guess())}")
print(f"constraint 2 evaluated at p1={p1_val}, p2={p2_val} is {prog.EvalBinding(constraint2, prog.initial_guess())}")
print(f"cost evaluated at p1={p1_val}, p2={p2_val} is {prog.EvalBinding(cost, prog.initial_guess())}")


## 제약 제거/완화(Removing/relaxing constraints)
솔버가 문제가 실현 불가능하다고 보고하는 경우, 실현 불가능한 제약 조건을 제거하거나 완화한 다음 문제를 다시 해결할 수 있다. 제약 조건을 제거하는 것은 간단하다. 처음에 제약 조건을 추가한 줄을 주석 처리하기만 하면 된다. 제약 조건 범위를 완화하려면 [UpdateLowerBound()](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.PyFunctionConstraint.UpdateLowerBound) 혹은 [UpdateUpperBound()](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.PyFunctionConstraint.UpdateUpperBound) 함수를 사용할 수 있다. 다음은 간단한 예입니다.

In [ ]:
# Relaxing the constraint
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)

# Add the constraint x^T * x <= 1
constraint1 = prog.AddConstraint(lambda z: [z.dot(z)], [0], [1], x)
constraint1.evaluator().set_description("inside unit circle")

# Add the constraint norm(x-[3, 0]) <= 1
constraint2 = prog.AddConstraint(lambda z: [np.sum((z - np.array([3, 0]))**2)], [0], [1], x)
constraint2.evaluator().set_description("distance to [3, 0] less than 1")

prog.SetInitialGuess(x, [1, 0])
solver = IpoptSolver()
result = solver.Solve(prog)
print(f"For the original problem, the solver status is {result.get_solution_result()}")
print(f"x is stuck at {result.GetSolution(x)}")
# Now get the infeasible constraint
infeasible_constraints = result.GetInfeasibleConstraints(prog)
for c in infeasible_constraints:
    print(f"infeasible constraint: {c}")

# Now update the upper bound of the first infeasible constraint
infeasible_constraints[0].evaluator().UpdateUpperBound([4])
# I also update the description of the constraint. Without updating the description, the
# problem still solves fine, but it would be confusing if you print out this constraint.
infeasible_constraints[0].evaluator().set_description("inside a circle with radius=2")
result = solver.Solve(prog)
print(f"For the relaxed problem, the solver status is {result.get_solution_result()}")
print(f"Solution is x = {result.GetSolution(x)}")